In [5]:
import os
env = os.environ
print("hello")
import os
env = os.environ

env['o9_web_api_url'] = 'https://mygenieazure.o9solutions.com'
env['o9_auth_token'] = '1bta34kxyvxcxpdqbuot30xw'
env['local_host'] = 'http://localhost:8000'
env['o9_tenant_name'] = 'qarelease_pyplatformhivetest'


print("done................")

from o9_common_utils.O9DataLake import O9DataLake, ResourceType, DataSource
query1 = "select (Version.[Version Name]*Product.[Product]*Time.FiscalWeek*SalesAccount.[Account]*Location.[Location]*{Measure.[DPSellOutUnitsActuals],Measure.[Mean Pricing Save PCT],Measure.[Placement Count],Measure.[Promotion Count],Measure.[DPSellOutPrice]});"
query2 = "select (Version.[Version Name]*Product.[Product]*Time.FiscalWeek*SalesAccount.[Account]*Location.[Location]*{Measure.[DPSellOutUnitsActuals],Measure.[Mean Pricing Save PCT],Measure.[Placement Count],Measure.[Promotion Count],Measure.[DPSellOutPrice]});"
O9DataLake.register("sales_df",DataSource.LS, ResourceType.IBPL, query1) 
O9DataLake.register("DPBaseInputs",DataSource.LS, ResourceType.IBPL, query2) 


hello
done................


In [9]:
help(O9DataLake.register)

Help on function register in module o9_common_utils.O9DataLake:

register(key, data_source=<DataSource.LS: 'liveserver'>, entity_type=<ResourceType.IBPL: 'ibpl_query'>, query=None)
    Usage :
    from o9_common_utils.O9DataLake import O9DataLake, ResourceTypes, DataSource
    O9DataLake.register("<input_df_name>",<datasource e.g. DataSource.LS>, <datasource e.g. ResourceType.IBPL>, query)



In [10]:


"""
exec plugin instance [CsvMGIP1_CsvMGIP2_CsvMGPrimDf_CsvMGOP1_Slice] for measures {[DPSellOutUnitsActuals]} using scope ([Product].[Product].Filter(#.Name in {[196426],[208821]}) * [Time].[Day] * [Version].[Version Name].[CurrentWorkingView]);

"""
import logging
from sklearn import tree

logger = logging.getLogger('o9_logger')

out_df = None
input_df = O9DataLake.get('sales_df')
predict_df = O9DataLake.get('DPBaseInputs')
if predict_df is not None and len(predict_df.index) > 0:
    x = input_df[['Mean Pricing Save PCT', 'Placement Count', 'Promotion Count', 'DPSellOutPrice']]
    y = input_df[['DPSellOutUnitsActuals']]
    x = x.fillna(0).values
    y = y.fillna(0).values

    clf = tree.DecisionTreeRegressor()
    clf = clf.fit(x, y)

    predict = predict_df[['Mean Pricing Save PCT', 'Placement Count', 'Promotion Count', 'DPSellOutPrice']].fillna(0).values
    # predict_keys = predict_df[['Time.[FiscalWeek]', 'Product.[Product]', 'Version.[Version Name]', 'SalesAccount.[Account]',                               'Location.[Location]']]
    # predict_keys["Product.[Product]"] = pd.to_numeric(predict_keys["Product.[Product]"], downcast='integer')
    predict_values = clf.predict(predict)
    predict_df["DPSellOutUnitsFcst"] = predict_values
    #out_df = pd.concat([predict_keys, out_df_part], axis=1)
    out_df=predict_df[['Time.[FiscalWeek]', 'Product.[Product]', 'Version.[Version Name]', 'SalesAccount.[Account]','Location.[Location]','DPSellOutUnitsFcst']]
    logger.info(out_df)

else:
    out_df = None

O9DataLake.put('DPBaseOutput1',out_df)

O9DataLake.put('DPBaseOutput2',out_df)



ModuleNotFoundError: No module named 'sklearn'